In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import networkx as nx
import pandas as pd
import re
sep = "---------"

today = str(datetime.now())

### Name of hOCR-file
hOCR = "_mdp.39015009351001-seq_27-37.html"

with open(r"data" + "\\" + hOCR, encoding='utf8') as fp:
    soup = BeautifulSoup(fp, 'html.parser')

## Initializing the graph

In [9]:
G = nx.MultiDiGraph()
G.graph['data'] = hOCR
G.graph['created'] = today
print(G.graph)

G.graph['id_head_count'] = 0
G.graph['id_header_count'] = 0
G.graph['id_text_count'] = 0
G.graph['id_year_count'] = 0
G.graph['id_office_count'] = 0
G.graph['id_magistrate_count'] = 0

{'data': '_mdp.39015009351001-seq_27-37.html', 'created': '2022-11-30 14:59:57.371994'}


## Fishing in beautiful soup for text areas per page

In [10]:
pages = soup.find_all("div", class_="ocr_page")

page_index = 0
print("Found " + str(len(pages)) + " pages")
print(sep)

for page_item in pages:
    areas = page_item.find_all("div", class_="ocr_carea")       
    page_index = page_index + 1
    print("Page " + str(page_index) + ": "+ str(len(areas)) + " text areas")
        
    for area_item in areas:
        heading_bool = False
        firstpara = area_item.p
        firstline = firstpara.span
        firstline_bbox_start = int(firstline["title"].split('bbox ')[1].split(' ')[0])
        
        if firstline_bbox_start > 1100:
            heading_bool = True
        
        
        firstline_words = firstline.find_all("span", class_="ocrx_word")

        string = ""
        for x in range(len(firstline_words)):
            string = string + " " + firstline_words[x].string
        
        if heading_bool == True:
            if string.__contains__('B.C.'):
                head = string
                head_id = "head_" + str(G.graph['id_head_count'])
                area_head = head_id
                G.graph['id_head_count'] = G.graph['id_head_count'] + 1
                G.add_node(head_id)
                G.nodes[head_id]['label'] = string
                print("Heading: " + area_head)
                
            else:
                area_header = string
                header_id = "header_" + str(G.graph['id_header_count'])
                area_header = header_id
                G.graph['id_header_count'] = G.graph['id_header_count'] + 1
                G.add_node(header_id)
                G.nodes[header_id]['label'] = string
                G.nodes[header_id]['head'] = area_head
                print("Heading: " + area_head + "| Header: " + area_header)
                
        print(string)
                
        #if heading_bool == False:
            

        

Found 11 pages
---------
Page 1: 4 text areas
Heading: head_0
 509 B.C. A.U.C. 245
Heading: head_0| Header: header_0
 Consuls
 L. Iunius M. f. - n. Brutus^1 Pat. (46a, Supb. 5.356ff.)
 1 Broughton ]
Page 2: 7 text areas
 2 509 B.C.
 (Liv. 2.6—7; Dion. Hal. 5.15—17; Plut. Popl. 9; cf. Auct. ad Herenn.
 Flor. 1.4.8; Dio fr. 13; Auct. Vir. Ill. 10.6; Eutrop. 1.10.1; Oros.
 L. Tarquinius Egeri f. Collatinus Pat. (8)
Heading: head_0| Header: header_1
 Consules Suffecti
 P. Valerius Volusi f. — n. Publicola Pat. (302) Cos. 508, 507,504
 Sp. Lucretius T.? f. — n. Tricipitinus Pat. (30)
Page 3: 13 text areas
 509 B.C. 3
 Dion. Hal. 5.19.2; Plut. Popl. 12.3; Eutrop. 1.10.2; Auct. Vir. Ill.
 M. Horatius M. f. —- n. Pulvillus Pat. (15) Cos. 507
Heading: head_0| Header: header_2
 Quaestors
 M. Minucius (Augurinus?) Pat. (17, cf. 32) Cos. 497, 491
 P. Veturius (Cicurinus Geminus?)^2 Pat. (*1) Cos. 499
Heading: head_0| Header: header_3
 Interrex
 Sp. Lucretius Tricipitinus Pat. (30) Cos. Suff. 509
H

In [4]:
node_list = list(G.nodes.data())

for node_list_item in node_list:
    node_attr_list = list(node_list_item)
    node_label = node_attr_list[1]['label']
    #node_head = node_attr_list[1]['head']
    print(node_attr_list)

print(len(G.nodes()))

['head_0', {'label': ' 509 B.C. A.U.C. 245'}]
['header_0', {'label': ' Consuls', 'head': 'head_0'}]
['header_1', {'label': ' Consules Suffecti', 'head': 'head_0'}]
['header_2', {'label': ' Quaestors', 'head': 'head_0'}]
['header_3', {'label': ' Interrex', 'head': 'head_0'}]
['header_4', {'label': ' Praefectus Urbi', 'head': 'head_0'}]
['header_5', {'label': ' Pontifices', 'head': 'head_0'}]
['header_6', {'label': ' Rex Sacrorum', 'head': 'head_0'}]
['head_1', {'label': ' 508 B.C. A.U.C. 246'}]
['header_7', {'label': ' Consuls', 'head': 'head_1'}]
['header_8', {'label': ' Legates, Ambassadors', 'head': 'head_1'}]
['header_9', {'label': ' Pontifices', 'head': 'head_1'}]
['head_2', {'label': ' 507 B.C. A.U.C. 247'}]
['header_10', {'label': ' Consuls^1', 'head': 'head_2'}]
['head_3', {'label': ' 506 B.C. A.U.C. 248'}]
['header_11', {'label': ' Consuls^1', 'head': 'head_3'}]
['head_4', {'label': ' 505 B.C. A.U.C. 249'}]
['header_12', {'label': ' Consuls', 'head': 'head_4'}]
['header_13', {'

In [ ]:
area = areas[1]
paras = area.find_all("p", class_="ocr_par")

para = paras[0]
firstline = para.span
firstline_bbox_start = int(firstline["title"].split('bbox ')[1].split(' ')[0])

if firstline_bbox_start > 2400:
    print("Heading starts at: " + str(firstline_bbox_start))
elif (firstline_bbox_start < 2400) and (firstline_bbox_start > 700):
    print("Year starts at: " + str(firstline_bbox_start))
else:
    print("Content starts at: " + str(firstline_bbox_start))

lines = para.find_all("span", class_="ocr_line")


In [ ]:
words = line.find_all("span", class_="ocrx_word")

string = ""
for x in range(len(words)):
    string = string + " " + words[x].string

print(string)